## Library

In [1]:
import torch
import torch.optim as optim
from torch.optim import Optimizer
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models

import numpy as np
import pydicom
import cv2
import os
import pandas as pd
from skimage.transform import resize

from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

## Init GPU

In [2]:
# Initialize GPU Device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)} is available.")
else:
    print("No GPU available. Training will run on CPU.")

print(device)

GPU: NVIDIA GeForce RTX 4070 SUPER is available.
cuda


In [3]:
%load_ext autoreload
%autoreload 2

## Config Info

In [4]:
# Constants
TEST_SIZE = 0.02
HEIGHT = 224
WIDTH = 224
CHANNELS = 3
TRAIN_BATCH_SIZE = 8
VALID_BATCH_SIZE = 8
TEST_BATCH_SIZE = 4
SHAPE = (HEIGHT, WIDTH, CHANNELS)

# Folders
DATA_DIR = './rsna-mil-training'

In [5]:
def preprocess_slice(slice, target_size=(224, 224)):
    slice = resize(slice, target_size, anti_aliasing=True)
    brain_channel = apply_windowing(slice, window=(40, 80))
    subdural_channel = apply_windowing(slice, window=(80, 200))
    bone_channel = apply_windowing(slice, window=(600, 2800))
    
    multichannel_slice = np.stack([brain_channel, subdural_channel, bone_channel], axis=-1)
    return multichannel_slice

def apply_windowing(slice, window):
    window_width, window_level = window
    lower_bound = window_level - window_width // 2
    upper_bound = window_level + window_width // 2
    
    windowed_slice = np.clip(slice, lower_bound, upper_bound)
    windowed_slice = (windowed_slice - lower_bound) / (upper_bound - lower_bound)
    return windowed_slice

In [6]:
def read_dicom_folder(folder_path):
    slices = []
    for filename in os.listdir(folder_path):
        if filename.endswith(".dcm"):
            file_path = os.path.join(folder_path, filename)
            ds = pydicom.dcmread(file_path)
            slices.append(ds.pixel_array)
    
    # Add black images if the number of slices is less than 60
    num_slices = len(slices)
    if num_slices < 60:
        black_slice = np.zeros_like(slices[0])
        for _ in range(60 - num_slices):
            slices.append(black_slice)
    
    return slices

In [7]:
def process_patient_data(data_dir, row):
    """
    Process data for a single patient based on the row from the DataFrame.
    
    Args:
        data_dir (str): The directory containing DICOM folders.
        row (pd.Series): A row from the patient_scan_labels DataFrame.

    Returns:
        Tuple: Preprocessed slices and label.
    """
    patient_id = row['patient_id'].replace('ID_', '')  # Remove 'ID_' prefix
    study_instance_uid = row['study_instance_uid'].replace('ID_', '')  # Remove 'ID_' prefix
    
    # Construct folder path based on patient_id and study_instance_uid
    folder_name = f"{patient_id}_{study_instance_uid}"
    folder_path = os.path.join(data_dir, folder_name)
    
    # Read and preprocess DICOM slices
    if os.path.exists(folder_path):
        slices = read_dicom_folder(folder_path)
        preprocessed_slices = [preprocess_slice(slice) for slice in slices]
        
        # Determine label based on any of the hemorrhage indicators
        label = 1 if row[['any', 'epidural', 'intraparenchymal', 'intraventricular', 'subarachnoid', 'subdural']].any() else 0
        
        return preprocessed_slices, label
    else:
        print(f"Folder not found: {folder_path}")
        return None, None  # Handle the case where the folder is not found

In [8]:
class TrainDatasetGenerator(Dataset):
    """
    A custom dataset class for training data.
    """
    def __init__(self, data_dir, patient_scan_labels):
        self.data_dir = data_dir
        self.patient_scan_labels = patient_scan_labels

    def __len__(self):
        return len(self.patient_scan_labels)

    def __getitem__(self, idx):
        row = self.patient_scan_labels.iloc[idx]
        preprocessed_slices, label = process_patient_data(self.data_dir, row)
        
        if preprocessed_slices is not None:
            # Convert the list of numpy arrays to a single numpy array
            preprocessed_slices = np.array(preprocessed_slices)  # Convert to numpy array
            return torch.tensor(preprocessed_slices, dtype=torch.float32), torch.tensor(label, dtype=torch.long)
        else:
            return None, None  # Handle the case where the folder is not found

class TestDatasetGenerator(Dataset):
    """
    A custom dataset class for testing data.
    """
    def __init__(self, data_dir, patient_scan_labels):
        self.data_dir = data_dir
        self.patient_scan_labels = patient_scan_labels

    def __len__(self):
        return len(self.patient_scan_labels)

    def __getitem__(self, idx):
        row = self.patient_scan_labels.iloc[idx]
        preprocessed_slices, label = process_patient_data(self.data_dir, row)
        
        if preprocessed_slices is not None:
            # Convert the list of numpy arrays to a single numpy array
            preprocessed_slices = np.array(preprocessed_slices)  # Convert to numpy array
            return torch.tensor(preprocessed_slices, dtype=torch.float32), torch.tensor(label, dtype=torch.long)
        else:
            return None, None  # Handle the case where the folder is not found
        

# Function to create DataLoader for training
def get_train_loader(data_dir, patient_scan_labels, batch_size=TRAIN_BATCH_SIZE, shuffle=True):
    train_dataset = TrainDatasetGenerator(data_dir, patient_scan_labels)
    return DataLoader(train_dataset, batch_size=batch_size, shuffle=shuffle)

# Function to create DataLoader for testing
def get_test_loader(data_dir, patient_scan_labels, batch_size=TEST_BATCH_SIZE):
    test_dataset = TestDatasetGenerator(data_dir, patient_scan_labels)
    return DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

## CNN Feature Extractor

In [9]:
class FeatureExtractor(torch.nn.Module):
    def __init__(self):
        super(FeatureExtractor, self).__init__()
        resnet18 = models.resnet18(pretrained=True)
        self.features = torch.nn.Sequential(*list(resnet18.children())[:-1])
        
    def forward(self, x):
        x = self.features(x)
        return x.view(x.size(0), -1)

In [10]:
class AttentionLayer(torch.nn.Module):
    def __init__(self, input_dim, attention_dim):
        super(AttentionLayer, self).__init__()
        self.attention = torch.nn.Sequential(
            torch.nn.Linear(input_dim, attention_dim),
            torch.nn.Tanh(),
            torch.nn.Linear(attention_dim, 1)
        )
    
    def forward(self, features):
        weights = self.attention(features)
        weights = torch.softmax(weights, dim=1)
        weighted_features = torch.sum(weights * features, dim=1)
        return weighted_features, weights

In [11]:
class MILModel(torch.nn.Module):
    def __init__(self, num_classes):
        super(MILModel, self).__init__()
        self.feature_extractor = FeatureExtractor()
        self.attention = AttentionLayer(512, 256)  # 512 is the output dimension of ResNet18
        # self.classifier = torch.nn.Linear(512, num_classes)
        self.classifier = torch.nn.Sequential(
            torch.nn.Linear(512, num_classes),
            torch.nn.Sigmoid()
        )
    def forward(self, x):
        batch_size, num_images, height, width, channels = x.size()
        x = x.view(-1, channels, height, width)
        features = self.feature_extractor(x)
        features = features.view(batch_size, num_images, -1)
        weighted_features, attention_weights = self.attention(features)
        output = self.classifier(weighted_features)
        return output, attention_weights

In [12]:
data_dir = '../rsna-mil-training'
patient_scan_labels = pd.read_csv('training_1000_scan_subset.csv')

In [13]:
train_loader = get_train_loader(data_dir, patient_scan_labels, batch_size=TRAIN_BATCH_SIZE)
test_loader = get_test_loader(data_dir, patient_scan_labels, batch_size=TEST_BATCH_SIZE)

In [14]:
num_epochs = 1
model = MILModel(num_classes=1).to(device)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
for epoch in range(num_epochs):
    print('############################################')
    print(f"Epoch [{epoch + 1}/{num_epochs}]")
    total_loss = 0
    correct = 0
    total = 0
    
    model.train()

    train_loader_tqdm = tqdm(train_loader, leave=False)
    for batch_idx, (batch_slices, batch_labels) in enumerate(train_loader_tqdm):
        try:
            # print(f"\nBatch {batch_idx + 1}")
            # print(f"Input shape: {batch_slices.shape}")
            # print(f"Labels shape: {batch_labels.shape}")
            # print(f"Labels: {batch_labels}")

            batch_slices = batch_slices.to(device)
            batch_labels = batch_labels.to(device)

            optimizer.zero_grad()
            outputs, attention_weights = model(batch_slices)
            
            # print(f"Model output shape: {outputs.shape}")
            # print(f"Model output: {outputs.squeeze()}")
            # print(f"Attention weights shape: {attention_weights.shape}")
            
            loss = criterion(outputs.squeeze(), batch_labels.float())
            # print(f"Loss: {loss.item()}")

            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            predicted = (outputs.squeeze() >= 0.5).float()
            correct += (predicted == batch_labels.float()).sum().item()
            total += batch_labels.size(0)

            batch_accuracy = (predicted == batch_labels.float()).float().mean().item()

            train_loader_tqdm.set_description(f"Epoch [{epoch + 1}/{num_epochs}]")
            train_loader_tqdm.set_postfix(loss=loss.item(), accuracy=batch_accuracy)

            # if batch_idx % 10 == 0:  # Print detailed info every 10 batches
            #     print(f"\nDetailed batch info:")
            #     print(f"Outputs: {outputs.squeeze()}")
            #     print(f"Labels: {batch_labels.float()}")
            #     print(f"Predictions: {predicted}")

        except Exception as e:
            print(f"Error in batch {batch_idx + 1}: {str(e)}")
            print(f"Input shape: {batch_slices.shape}")
            print(f"Labels shape: {batch_labels.shape}")
            raise  # Re-raise the exception for full traceback

    avg_loss = total_loss / len(train_loader)
    accuracy = correct / total

    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {avg_loss:.4f}, Accuracy: {accuracy:.4f}')

/home/hskha23/miniconda3/envs/tf/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/hskha23/miniconda3/envs/tf/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


############################################
Epoch [1/1]


Epoch [1/1], Loss: 0.7056, Accuracy: 0.5450
